In [1]:
import pymysql
import pandas as pd
import numpy as np

In [2]:
conn=pymysql.connect(host='mysql-backup.shanzhen.me',
                     user='readonly_medicine_liuqq',
                     password="r3g1^QWgV8GeYJw4",
                     database='asgard_dataplatform',
                     port= 3001,
                     charset='utf8')

In [16]:
thyroid = pd.read_excel(r"C:\Users\13670\Desktop\待完成\项目4 回声tag分析\甲状腺\20201014\甲状腺dict.xlsx")

In [4]:
id_list1 = thyroid.loc[thyroid['logic']== "B" ,'tag_id']
id_list2 = thyroid.loc[thyroid['logic']== "b" ,'tag_id']

In [5]:
id_list = pd.concat([id_list1,id_list2])
id_list

12    OBDIM0074Tag0007
13    OBDIM0074Tag0008
14    OBDIM0074Tag0009
19    OBDIM0074Tag0013
20    OBDIM0074Tag0014
Name: tag_id, dtype: object

In [6]:
tag_df = pd.DataFrame(columns=['ORDER_CODE', 'TAG_ID'])

In [7]:
for items in id_list:
    id_str = "'"+ items + "'"
    tagsql = 'SELECT ORDER_CODE,TAG_ID \
    FROM t66_apollo_offline_analysis_tag \
    WHERE SOURCE_TYPE = 1 \
    AND SZ_STATUS = 1 AND TAG_ID IN ('+id_str+') '
    tag = pd.read_sql(tagsql,con=conn)
    tag_df = pd.concat([tag_df,tag],axis = 0) 
    
tag_df  

,ORDER_CODE,TAG_ID
0,000275084773,OBDIM0074Tag0007
1,385950161146,OBDIM0074Tag0007
2,000319342337,OBDIM0074Tag0007
3,386150125159,OBDIM0074Tag0007
4,000457704610,OBDIM0074Tag0007
...,...,...
9635,385043612897,OBDIM0074Tag0014
9636,385126084523,OBDIM0074Tag0014
9637,385127569434,OBDIM0074Tag0014
9638,385487364214,OBDIM0074Tag0014


In [8]:
tag_df.to_pickle(r"C:\Users\13670\Desktop\待完成\项目4 回声tag分析\甲状腺\整体性病变\tagall.pkl")

In [9]:
dict_ = thyroid.set_index('tag_id').to_dict()['abbr']

In [10]:
tag_df.replace(dict_,inplace=True)

In [17]:
desclist = thyroid.loc[thyroid['logic']== "B",'abbr'].tolist()
summlist = thyroid.loc[thyroid['logic']=="b",'abbr'].tolist()

In [18]:
DescDf = tag_df.loc[tag_df['TAG_ID'].isin(desclist)].drop_duplicates()
SummDf = tag_df.loc[tag_df['TAG_ID'].isin(summlist)].drop_duplicates()

In [19]:
SummDf

,ORDER_CODE,TAG_ID
0,385921352557,弥漫性改变
1,000302860061,弥漫性改变
2,000457704610,弥漫性改变
3,386215558433,弥漫性改变
4,000576682542,弥漫性改变
...,...,...
9635,385043612897,弥漫性改变
9636,385126084523,弥漫性改变
9637,385127569434,弥漫性改变
9638,385487364214,弥漫性改变


In [13]:
def ExhaustPatterns(DescDf,SummDf):
    ordercode=[]
    desclist=[]
    summlist=[]
    for item in DescDf['ORDER_CODE'].drop_duplicates():
        desclistsub=DescDf.loc[DescDf['ORDER_CODE']== item,'TAG_ID'].tolist()
        summlistsub=SummDf.loc[SummDf['ORDER_CODE']== item,'TAG_ID'].tolist()
        ordercode.append(item)
        desclist.append(desclistsub)
        summlist.append(summlistsub)
    TagOrderCode=pd.DataFrame({'ORDER_CODE':ordercode,'desclist':desclist,'summlist':summlist}) 
    return TagOrderCode

TagOrderCode = ExhaustPatterns(DescDf,SummDf)

In [20]:
TagOrderCode1 = ExhaustPatterns(DescDf,SummDf)

In [14]:
TagOrderCode

,ORDER_CODE,desclist,summlist
0,000275084773,"[回声增粗, 回声不均匀]",[]
1,385950161146,"[回声增粗, 回声不均匀]",[]
2,000319342337,"[回声增粗, 回声不均匀]",[]
3,386150125159,"[回声增粗, 回声不均匀]",[]
4,000457704610,"[回声增粗, 回声不均匀]",[弥漫性改变]
...,...,...,...
43161,385612688055,[回声不均匀],[]
43162,385624563750,[回声不均匀],[]
43163,385675770726,[回声不均匀],[]
43164,385688053969,[回声不均匀],[]


In [15]:
df = TagOrderCode.applymap(lambda x: tuple(x))
df_tuple = pd.DataFrame(df.groupby(['desclist','summlist'])['ORDER_CODE'].count()).reset_index()
df_tuple

,desclist,summlist,ORDER_CODE
0,"(回声不均匀,)",(),29540
1,"(回声不均匀,)","(增大,)",525
2,"(回声不均匀,)","(增大, 弥漫性改变)",227
3,"(回声不均匀,)","(弥漫性改变,)",2949
4,"(回声增强,)",(),61
5,"(回声增强,)","(弥漫性改变,)",1
6,"(回声增强, 回声不均匀)",(),59
7,"(回声增强, 回声不均匀)","(增大,)",5
8,"(回声增强, 回声不均匀)","(增大, 弥漫性改变)",1
9,"(回声增强, 回声不均匀)","(弥漫性改变,)",5


In [21]:
df1 = TagOrderCode1.applymap(lambda x: tuple(x))
df_tuple1 = pd.DataFrame(df1.groupby(['desclist','summlist'])['ORDER_CODE'].count()).reset_index()
df_tuple1

,desclist,summlist,ORDER_CODE
0,"(回声不均匀,)",(),29540
1,"(回声不均匀,)","(弥漫性改变,)",2949
2,"(回声不均匀, 增大)",(),525
3,"(回声不均匀, 增大)","(弥漫性改变,)",227
4,"(回声增强,)",(),61
5,"(回声增强,)","(弥漫性改变,)",1
6,"(回声增强, 回声不均匀)",(),59
7,"(回声增强, 回声不均匀)","(弥漫性改变,)",5
8,"(回声增强, 回声不均匀, 增大)",(),5
9,"(回声增强, 回声不均匀, 增大)","(弥漫性改变,)",1
